# Plotly Interactive Charts in Trelliscope

This notebook demonstrates **fully interactive Plotly visualizations** in the Trelliscope Dash viewer.

## Features Demonstrated

- ✅ **Native Plotly Rendering** - No iframes, true `dcc.Graph` components
- ✅ **Full Interactivity** - Hover tooltips, zoom, pan, select
- ✅ **Responsive Design** - Charts resize with layout changes
- ✅ **Multiple Chart Types** - Line, scatter, bar, heatmap, box plots
- ✅ **Advanced Plotly Features** - Subplots, annotations, custom styling

## Chart Types Included

1. **Time Series** - Interactive line charts with hover details
2. **Scatter Plots** - Bubble charts with color/size encoding
3. **Bar Charts** - Comparative analysis with tooltips
4. **Box Plots** - Statistical distributions
5. **Heatmaps** - Correlation matrices
6. **Subplots** - Multiple views in one panel

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import sys

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from trelliscope import Display
from trelliscope.dash_viewer import create_dash_app

In [ ]:
# Generate synthetic refinery margins data
from datetime import datetime, timedelta

countries = ['Algeria', 'Denmark', 'Germany', 'Japan', 'United Kingdom']
n_days = 180
start_date = datetime(2023, 1, 1)

data_rows = []
np.random.seed(42)

for country in countries:
    dates = [start_date + timedelta(days=i) for i in range(n_days)]
    
    # Generate realistic refinery and crude data with trends
    base_refinery = np.random.uniform(500, 2000)
    base_crude = np.random.uniform(400, 1800)
    
    trend_ref = np.linspace(0, np.random.uniform(-100, 100), n_days)
    trend_crude = np.linspace(0, np.random.uniform(-100, 100), n_days)
    
    seasonality_ref = 100 * np.sin(np.linspace(0, 4*np.pi, n_days))
    seasonality_crude = 80 * np.sin(np.linspace(0, 4*np.pi, n_days))
    
    noise_ref = np.random.randn(n_days) * 50
    noise_crude = np.random.randn(n_days) * 40
    
    refinery_kbd = base_refinery + trend_ref + seasonality_ref + noise_ref
    crude_kbd = base_crude + trend_crude + seasonality_crude + noise_crude
    
    for i in range(n_days):
        data_rows.append({
            'country': country,
            'date': dates[i],
            'refinery_kbd': max(0, refinery_kbd[i]),
            'crude_kbd': max(0, crude_kbd[i])
        })

df = pd.DataFrame(data_rows)

print(f"Generated {len(df):,} rows")
print(f"\nCountries: {', '.join(sorted(df['country'].unique()))}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")

## Create Visualization Functions

Each function creates a different Plotly chart type with full interactivity.

In [ ]:
def create_time_series_chart(data, country):
    """
    Interactive time series with dual y-axes.
    
    Features: hover details, zoom, pan, range selector
    """
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Refinery Capacity', 'Crude Imports'),
        vertical_spacing=0.12,
        row_heights=[0.6, 0.4]
    )
    
    # Refinery capacity
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['refinery_kbd'],
            mode='lines',
            name='Refinery',
            line=dict(color='#2c7fb8', width=2),
            fill='tozeroy',
            fillcolor='rgba(44, 127, 184, 0.2)',
            hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Capacity: %{y:.1f} kbd<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Crude imports
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['crude_kbd'],
            mode='lines',
            name='Crude Imports',
            line=dict(color='#d95f02', width=2),
            hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Imports: %{y:.1f} kbd<extra></extra>'
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title=f"{country} - Energy Production",
        showlegend=False,
        hovermode='x unified',
        height=500,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=60, r=20, t=80, b=40)
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0')
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig


def create_scatter_chart(data, country):
    """
    Bubble chart with color gradient.
    
    Features: hover details, pan, zoom, color scale
    """
    fig = go.Figure()
    
    # Calculate year for coloring
    data = data.copy()
    data['year'] = data['date'].dt.year
    
    fig.add_trace(
        go.Scatter(
            x=data['refinery_kbd'],
            y=data['crude_kbd'],
            mode='markers',
            marker=dict(
                size=8,
                color=data['year'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Year"),
                line=dict(width=0.5, color='white')
            ),
            text=data['date'].dt.strftime('%Y-%m-%d'),
            hovertemplate='<b>%{text}</b><br>' +
                         'Refinery: %{x:.1f} kbd<br>' +
                         'Crude: %{y:.1f} kbd<extra></extra>'
        )
    )
    
    fig.update_layout(
        title=f"{country} - Refinery vs Crude Imports",
        xaxis_title='Refinery Capacity (kbd)',
        yaxis_title='Crude Imports (kbd)',
        hovermode='closest',
        height=450,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0', zeroline=True)
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0', zeroline=True)
    
    return fig


def create_bar_chart(data, country):
    """
    Grouped bar chart with error bands.
    
    Features: hover details, tooltips, color coding
    """
    # Aggregate by year
    yearly = data.copy()
    yearly['year'] = yearly['date'].dt.year
    
    agg = yearly.groupby('year').agg({
        'refinery_kbd': ['mean', 'std'],
        'crude_kbd': ['mean', 'std']
    }).reset_index()
    
    agg.columns = ['year', 'ref_mean', 'ref_std', 'crude_mean', 'crude_std']
    
    fig = go.Figure()
    
    # Refinery bars
    fig.add_trace(
        go.Bar(
            x=agg['year'],
            y=agg['ref_mean'],
            name='Refinery',
            marker_color='#2c7fb8',
            error_y=dict(type='data', array=agg['ref_std'], visible=True),
            hovertemplate='<b>Year %{x}</b><br>Avg: %{y:.1f} kbd<extra></extra>'
        )
    )
    
    # Crude bars
    fig.add_trace(
        go.Bar(
            x=agg['year'],
            y=agg['crude_mean'],
            name='Crude Imports',
            marker_color='#d95f02',
            error_y=dict(type='data', array=agg['crude_std'], visible=True),
            hovertemplate='<b>Year %{x}</b><br>Avg: %{y:.1f} kbd<extra></extra>'
        )
    )
    
    fig.update_layout(
        title=f"{country} - Annual Averages",
        xaxis_title='Year',
        yaxis_title='Capacity (kbd)',
        barmode='group',
        height=450,
        plot_bgcolor='white',
        paper_bgcolor='white',
        legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
    )
    
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig


def create_box_plot(data, country):
    """
    Statistical box plot with outliers.
    
    Features: hover details, quartiles, outlier detection
    """
    fig = go.Figure()
    
    fig.add_trace(
        go.Box(
            y=data['refinery_kbd'],
            name='Refinery',
            marker_color='#2c7fb8',
            boxmean='sd',
            hovertemplate='%{y:.1f} kbd<extra></extra>'
        )
    )
    
    fig.add_trace(
        go.Box(
            y=data['crude_kbd'],
            name='Crude Imports',
            marker_color='#d95f02',
            boxmean='sd',
            hovertemplate='%{y:.1f} kbd<extra></extra>'
        )
    )
    
    fig.update_layout(
        title=f"{country} - Distribution Analysis",
        yaxis_title='Capacity (kbd)',
        height=450,
        showlegend=True,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig


print("✓ Created 4 visualization functions")

## Generate Interactive Plotly Panels

Create multiple chart types for each country.

In [ ]:
countries = sorted(df['country'].unique())
chart_types = [
    ('Time Series', create_time_series_chart),
    ('Scatter', create_scatter_chart),
    ('Bar Chart', create_bar_chart),
    ('Box Plot', create_box_plot)
]

print(f"Generating {len(countries)} countries × {len(chart_types)} chart types = {len(countries) * len(chart_types)} panels...\n")

panels_data = []

for country in countries:
    country_df = df[df['country'] == country].copy()
    
    # Calculate statistics
    stats = {
        'country': country,
        'avg_refinery': country_df['refinery_kbd'].mean(),
        'avg_crude': country_df['crude_kbd'].mean(),
        'n_observations': len(country_df),
        'start_date': country_df['date'].min(),
        'end_date': country_df['date'].max()
    }
    
    # Create each chart type
    for chart_name, chart_func in chart_types:
        fig = chart_func(country_df, country)
        
        panels_data.append({
            **stats,
            'chart_type': chart_name,
            'panel': fig
        })
    
    print(f"  ✓ {country}: {len(chart_types)} chart types")

panels_df = pd.DataFrame(panels_data)
print(f"\n✓ Generated {len(panels_df)} interactive Plotly panels")

In [ ]:
# Preview data structure
print("DataFrame shape:", panels_df.shape)
print("\nColumns:", list(panels_df.columns))
print("\nFirst few rows:")
print(panels_df[['country', 'chart_type', 'avg_refinery', 'avg_crude']].head(8))

## Create Trelliscope Display

The viewer will extract Plotly figures from HTML and render them **natively as interactive `dcc.Graph` components**.

In [ ]:
display = (
    Display(
        panels_df,
        name="plotly_interactive_showcase",
        description="Interactive Plotly Charts - Time Series, Scatter, Bar, Box Plots"
    )
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=3, nrow=2)
    .set_default_labels(["country", "chart_type", "avg_refinery"])
)

print("✓ Display configured with Plotly panels")
print(f"  Name: {display.name}")
print(f"  Panels: {len(panels_df)}")
print(f"  Default layout: 3 columns × 2 rows")

## Launch Interactive Viewer

### 🎯 Features to Test

**Plotly Interactivity:**
- ✨ **Hover** - Move mouse over charts to see detailed tooltips
- 🔍 **Zoom** - Click and drag to zoom into regions
- 🖱️ **Pan** - Shift + drag to pan around
- 📊 **Toolbar** - Use Plotly modebar (zoom, pan, reset, download)
- 🎨 **Responsive** - Adjust grid layout and watch charts resize

**Filtering & Sorting:**
- 🌍 Filter by **country** (multi-select)
- 📈 Filter by **chart_type** (select specific visualizations)
- 🔢 Filter by **avg_refinery** (range slider)
- ↕️ Sort by any metric

**Keyboard Shortcuts:**
- `←` / `→` - Previous/Next page
- `+` / `-` - Increase/Decrease grid size
- `/` - Focus search
- `?` - Show keyboard shortcuts

The viewer will open at http://localhost:8054

In [ ]:
app = create_dash_app(display)
app.run(debug=False, host='127.0.0.1', port=8054)